**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [2]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [3]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [4]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [5]:
df_supply

,0
0,365
1,355
2,346
3,694
4,799


In [6]:
df_demand

,0
0,826
1,268
2,568
3,587
4,107


In [7]:
df_ori_dest

,0,1,2,3,4
0,87.987960,93.765322,186.947526,104.115745,67.618090
1,90.007134,89.091878,181.966955,107.352800,70.458703
2,88.449596,90.980027,184.212746,105.360680,68.587755
3,87.739254,91.838635,185.232177,104.447890,67.735752
4,88.206881,94.469172,187.538990,104.113869,67.706097


In [8]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [9]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

2559 2356


In [10]:
m = gp.Model("probrema")

In [11]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [12]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [13]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [14]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [15]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [16]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [17]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45 rows, 100 columns and 350 nonzeros
Model fingerprint: 0x7f2f4f9c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 15 rows and 0 columns
Presolve time: 0.01s
Presolved: 30 rows, 100 columns, 275 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.945000e+02   0.000000e+00      0s
      24    1.7972796e+05   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.797279609e+05


In [18]:
m.ObjVal

179727.96086490253